In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import log_loss

In [2]:
#import in the data 
train_df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
test= test_df.drop("id",axis=1)

In [3]:
#prepare the data for modeling 
label_dict = {"Class_1":0,"Class_2":1,"Class_3":2,"Class_4":3}

X = train_df.drop(['id', 'target'], axis=1)
y = train_df['target'].apply(lambda x : label_dict[x])

In [4]:
losses = []
sk_fold = StratifiedKFold(5)
clf = CatBoostClassifier(verbose=False)
for train_idx,val_idx in sk_fold.split(X,y):
        x_train,x_valid = X.iloc[train_idx],X.iloc[val_idx]
        y_train,y_valid = y.iloc[train_idx],y.iloc[val_idx]
        clf.fit(x_train,y_train,eval_set=[(x_valid, y_valid)],early_stopping_rounds=100)
        preds = clf.predict_proba(x_valid)
        loss = log_loss(y_valid,preds)
        losses.append(loss)
        print(loss)
 

1.0951175348921185
1.0948789826908198
1.0896197731417563
1.0940115213330792
1.092319008077403


In [5]:
print(np.average(losses))

1.0931893640270354


In [6]:
clf.fit(X,y,early_stopping_rounds=100)
preds = clf.predict_proba(test)

In [7]:
submission_df = pd.DataFrame(preds,columns=['Class_1', 'Class_2', 'Class_3','Class_4'])
submission_df.insert(0, 'id', test_df['id'])

In [8]:
submission_df.head()

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.077165,0.596047,0.205985,0.120802
1,100001,0.072078,0.687866,0.145110,0.094946
2,100002,0.078835,0.637798,0.170178,0.113190
3,100003,0.084198,0.483451,0.292797,0.139554
4,100004,0.070225,0.616591,0.199242,0.113941


In [9]:
submission_df.to_csv('submission.csv', index=False)